In [1]:
# !unzip /content/full\ graphs.zip


In [2]:
# !pip install torch_geometric
# import os
# import torch
# import pickle
# import pandas as pd

# def load_graph(path, is_pickle=True):
#     """
#     Load a molecule graph (.pkl) or a protein graph (.pt).
#     If is_pickle is True, use pickle to load the file; otherwise, use torch.load.
#     """
#     if is_pickle:
#         with open(path, 'rb') as f:
#             return pickle.load(f)
#     else:
#         return torch.load(path)

# def prepare_dataset_individual_save_as_pt(filtered_dataset, molecule_graph_dir, protein_graph_dir, output_dir):
#     """
#     Incrementally prepares the dataset and saves each (molecule, protein, target) tuple as a separate .pt file.

#     Args:
#     - filtered_dataset: The filtered KIBA dataset (DataFrame).
#     - molecule_graph_dir: Directory where molecule graphs are stored.
#     - protein_graph_dir: Directory where protein graphs are stored.
#     - output_dir: Directory to save the prepared dataset incrementally.
#     """
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     for index, row in filtered_dataset.iterrows():
#         protein_id = row['Target_ID']
#         chembl_id = row['Drug_ID']

#         # Load the protein graph (.pt)
#         pro_graph_path = os.path.join(protein_graph_dir, f"{protein_id}_graph.pt")
#         if not os.path.exists(pro_graph_path):
#             print(f"Protein graph not found: {protein_id}")
#             continue
#         pro_graph = load_graph(pro_graph_path, is_pickle=False)

#         # Load the molecule graph (.pkl)
#         mol_graph_path = os.path.join(molecule_graph_dir, f"{chembl_id}_graph.pkl")
#         if not os.path.exists(mol_graph_path):
#             print(f"Molecule graph not found: {chembl_id}")
#             continue
#         mol_graph = load_graph(mol_graph_path)

#         # Load target (affinity value)
#         target = torch.tensor([row['Y']], dtype=torch.float)

#         # Create the sample as a tuple (molecule graph, protein graph, target)
#         sample = (mol_graph, pro_graph, target)

#         # Save the sample as a .pt file
#         sample_path = os.path.join(output_dir, f"sample_{index}.pt")
#         torch.save(sample, sample_path)

#         print(f"Saved sample {index} as {sample_path}")

# # Example usage for individual saving
# molecule_graph_dir = '/content/molecule_graphs'  # Directory where molecule graphs are stored
# protein_graph_dir = '/content/ProteinGraphs'  # Directory where protein graphs are stored
# filtered_dataset_path = '/content/filtered_KibaDataSet.csv'  # Path to the filtered dataset CSV
# output_dir = '/content/prepared_samples/'  # Directory to save individual samples

# # Load filtered dataset CSV
# filtered_dataset = pd.read_csv(filtered_dataset_path)

# # Prepare the dataset incrementally, saving each sample as a .pt file
# prepare_dataset_individual_save_as_pt(filtered_dataset, molecule_graph_dir, protein_graph_dir, output_dir)

# print("Dataset preparation completed.")


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, global_max_pool as gmp, global_add_pool as gap,global_mean_pool as gep,global_sort_pool
from torch_geometric.utils import dropout_adj


# GCN based model
class GNNNet(torch.nn.Module):
    def __init__(self, n_output=1, num_features_pro=54, num_features_mol=78, output_dim=128, dropout=0.2):
        super(GNNNet, self).__init__()

        print('GNNNet Loaded')
        self.n_output = n_output
        self.mol_conv1 = GCNConv(num_features_mol, num_features_mol)
        self.mol_conv2 = GCNConv(num_features_mol, num_features_mol * 2)
        self.mol_conv3 = GCNConv(num_features_mol * 2, num_features_mol * 4)
        self.mol_fc_g1 = torch.nn.Linear(num_features_mol * 4, 1024)
        self.mol_fc_g2 = torch.nn.Linear(1024, output_dim)

        # self.pro_conv1 = GCNConv(embed_dim, embed_dim)
        self.pro_conv1 = GCNConv(num_features_pro, num_features_pro)
        self.pro_conv2 = GCNConv(num_features_pro, num_features_pro * 2)
        self.pro_conv3 = GCNConv(num_features_pro * 2, num_features_pro * 4)
        # self.pro_conv4 = GCNConv(embed_dim * 4, embed_dim * 8)
        self.pro_fc_g1 = torch.nn.Linear(num_features_pro * 4, 1024)
        self.pro_fc_g2 = torch.nn.Linear(1024, output_dim)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

        # combined layers
        self.fc1 = nn.Linear(2 * output_dim, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.out = nn.Linear(512, self.n_output)

    def forward(self, data_mol, data_pro):
        # get graph input
        mol_x, mol_edge_index, mol_batch = data_mol.x, data_mol.edge_index, data_mol.batch
        # get protein input
        target_x, target_edge_index, target_batch = data_pro.x, data_pro.edge_index, data_pro.batch

        # target_seq=data_pro.target

        # print('size')
        # print('mol_x', mol_x.size(), 'edge_index', mol_edge_index.size(), 'batch', mol_batch.size())
        # print('target_x', target_x.size(), 'target_edge_index', target_batch.size(), 'batch', target_batch.size())

        x = self.mol_conv1(mol_x, mol_edge_index)
        x = self.relu(x)

        # mol_edge_index, _ = dropout_adj(mol_edge_index, training=self.training)
        x = self.mol_conv2(x, mol_edge_index)
        x = self.relu(x)

        # mol_edge_index, _ = dropout_adj(mol_edge_index, training=self.training)
        x = self.mol_conv3(x, mol_edge_index)
        x = self.relu(x)
        x = gep(x, mol_batch)  # global pooling

        # flatten
        x = self.relu(self.mol_fc_g1(x))
        x = self.dropout(x)
        x = self.mol_fc_g2(x)
        x = self.dropout(x)

        xt = self.pro_conv1(target_x, target_edge_index)
        xt = self.relu(xt)

        # target_edge_index, _ = dropout_adj(target_edge_index, training=self.training)
        xt = self.pro_conv2(xt, target_edge_index)
        xt = self.relu(xt)

        # target_edge_index, _ = dropout_adj(target_edge_index, training=self.training)
        xt = self.pro_conv3(xt, target_edge_index)
        xt = self.relu(xt)

        # xt = self.pro_conv4(xt, target_edge_index)
        # xt = self.relu(xt)
        xt = gep(xt, target_batch)  # global pooling

        # flatten
        xt = self.relu(self.pro_fc_g1(xt))
        xt = self.dropout(xt)
        xt = self.pro_fc_g2(xt)
        xt = self.dropout(xt)

        # print(x.size(), xt.size())
        # concat
        xc = torch.cat((x, xt), 1)
        # add some dense layers
        xc = self.fc1(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        xc = self.fc2(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        out = self.out(xc)
        return out


In [9]:
# ... [Your existing imports and GNNNet class definition remain unchanged] ...

import os
import torch
import torch.optim as optim
from torch.nn import MSELoss
from sklearn.model_selection import KFold
import numpy as np
from tqdm import tqdm
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv, global_mean_pool as gep
from scipy.stats import pearsonr
import warnings
import itertools

# Suppress FutureWarning related to torch.load
warnings.filterwarnings('ignore', category=FutureWarning)

def adjust_state_dict(state_dict, model_state_dict):
    new_state_dict = {}
    for k in state_dict.keys():
        new_k = k
        # Adjust mol_conv layers
        if 'mol_conv' in k:
            if 'weight' in k and 'lin.weight' not in k:
                new_k = k.replace('weight', 'lin.weight')
            if 'bias' in k and 'lin.bias' not in k:
                new_k = k.replace('bias', 'lin.bias')
        # Adjust pro_conv layers
        elif 'pro_conv' in k:
            if 'weight' in k and 'lin.weight' not in k:
                new_k = k.replace('weight', 'lin.weight')
            if 'bias' in k and 'lin.bias' not in k:
                new_k = k.replace('bias', 'lin.bias')

        param = state_dict[k]
        if new_k in model_state_dict:
            model_param = model_state_dict[new_k]
            if param.shape != model_param.shape:
                if len(param.shape) == 2:
                    # Transpose the weight matrix
                    param = param.t()
                    if param.shape != model_param.shape:
                        print(f"Shape mismatch after transpose for {new_k}: checkpoint {param.shape}, model {model_param.shape}")
                else:
                    print(f"Shape mismatch for {new_k}: checkpoint {param.shape}, model {model_param.shape}")
        new_state_dict[new_k] = param
    return new_state_dict

# Define the load_sample function
def load_sample(path):
    # Load individual sample from file
    sample = torch.load(path)
    mol_data = sample[0]
    pro_data = sample[1]
    target = sample[2]

    # Convert dictionaries to Data objects if necessary
    if isinstance(mol_data, dict):
        mol_data = Data(**mol_data)
    if isinstance(pro_data, dict):
        pro_data = Data(**pro_data)

    # Ensure that 'x' attribute is set
    if not hasattr(mol_data, 'x') or mol_data.x is None:
        if hasattr(mol_data, 'features'):
            mol_data.x = mol_data.features
            del mol_data.features
        else:
            raise ValueError("mol_data does not have 'x' or 'features' attribute")

    if not hasattr(pro_data, 'x') or pro_data.x is None:
        if hasattr(pro_data, 'features'):
            pro_data.x = pro_data.features
            del pro_data.features
        else:
            raise ValueError("pro_data does not have 'x' or 'features' attribute")

    # Ensure 'x' is a float tensor
    if not isinstance(mol_data.x, torch.Tensor):
        mol_data.x = torch.tensor(mol_data.x)
    if not isinstance(pro_data.x, torch.Tensor):
        pro_data.x = torch.tensor(pro_data.x)

    if mol_data.x.dtype != torch.float:
        mol_data.x = mol_data.x.float()
    if pro_data.x.dtype != torch.float:
        pro_data.x = pro_data.x.float()

    # Adjust 'edge_index' for mol_data
    # Ensure 'edge_index' is a tensor of type torch.long
    if not isinstance(mol_data.edge_index, torch.Tensor):
        mol_data.edge_index = torch.tensor(mol_data.edge_index, dtype=torch.long)
    else:
        mol_data.edge_index = mol_data.edge_index.long()

    # Ensure 'edge_index' has shape [2, num_edges]
    if mol_data.edge_index.shape[0] != 2:
        mol_data.edge_index = mol_data.edge_index.t()

    # Adjust 'edge_index' for pro_data
    if not isinstance(pro_data.edge_index, torch.Tensor):
        pro_data.edge_index = torch.tensor(pro_data.edge_index, dtype=torch.long)
    else:
        pro_data.edge_index = pro_data.edge_index.long()

    if pro_data.edge_index.shape[0] != 2:
        pro_data.edge_index = pro_data.edge_index.t()

    # Set 'num_nodes' attribute to suppress warnings
    mol_data.num_nodes = mol_data.x.size(0)
    pro_data.num_nodes = pro_data.x.size(0)

    return (mol_data, pro_data, target)

# Define the batch_loader function
def batch_loader(file_list, sample_dir, batch_size):
    batch = []
    for idx, file_name in enumerate(file_list):
        sample_path = os.path.join(sample_dir, file_name)
        sample = load_sample(sample_path)
        batch.append(sample)
        if len(batch) == batch_size:
            yield batch
            batch = []
    if len(batch) > 0:
        yield batch

# Define the evaluation metrics functions
def get_mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def get_ci(y_true, y_pred):
    """
    Compute the concordance index between true and predicted values.
    """
    pairs = itertools.combinations(range(len(y_true)), 2)
    c = 0
    s = 0
    for i, j in pairs:
        if y_true[i] != y_true[j]:
            s += 1
            if (y_true[i] < y_true[j] and y_pred[i] < y_pred[j]) or \
               (y_true[i] > y_true[j] and y_pred[i] > y_pred[j]):
                c += 1
            elif y_pred[i] == y_pred[j]:
                c += 0.5
    return c / s if s != 0 else 0

def get_pearson(y_true, y_pred):
    return pearsonr(y_true.flatten(), y_pred.flatten())[0]



def train_5fold_cross_validation(sample_dir, num_epochs=1000, n_splits=5, lr=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Running on {device}.")

    sample_files = [f for f in os.listdir(sample_dir) if f.endswith('.pt')]
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Create a single directory for all checkpoints
    training_model_dir = os.path.join(sample_dir, 'TrainingModel')
    if not os.path.exists(training_model_dir):
        os.makedirs(training_model_dir)
        print(f"Created directory for checkpoints at {training_model_dir}")
    else:
        print(f"Using existing TrainingModel directory at {training_model_dir}")

    results = []
    loss_fn = MSELoss()

    for fold, (train_idx, test_idx) in enumerate(kfold.split(sample_files)):
        fold_number = fold + 1
        print(f'\nFold {fold_number}/{n_splits}')
        train_files = [sample_files[i] for i in train_idx]
        test_files = [sample_files[i] for i in test_idx]

        # Determine input feature dimensions from your data
        sample = load_sample(os.path.join(sample_dir, train_files[0]))
        mol_data = sample[0]
        pro_data = sample[1]

        num_features_mol = mol_data.x.size(1)
        num_features_pro = pro_data.x.size(1)

        # Initialize the GNN model with correct input dimensions
        model = GNNNet(
            num_features_mol=num_features_mol,
            num_features_pro=num_features_pro
        ).to(device)
        print(f"Model is on device: {next(model.parameters()).device}")

        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Initialize starting epoch
        start_epoch = 1

        # Check for existing checkpoints in TrainingModel directory for the current fold
        existing_checkpoints = [f for f in os.listdir(training_model_dir)
                                if f.endswith('.pt') and f.startswith(f'model_fold{fold_number}_epoch')]

        if existing_checkpoints:
            # Find the latest checkpoint based on epoch number
            latest_checkpoint = max(existing_checkpoints,
                                    key=lambda x: int(x.split('_epoch')[1].split('.pt')[0]))
            checkpoint_path = os.path.join(training_model_dir, latest_checkpoint)
            print(f"Loading checkpoint for fold {fold_number} from {checkpoint_path}")
            checkpoint = torch.load(checkpoint_path, map_location=device)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            loaded_epoch = checkpoint['epoch']
            start_epoch = loaded_epoch + 1
            print(f"Resuming training from epoch {start_epoch}")
        else:
            print(f"No checkpoint found for fold {fold_number}, loading pretrained model from 'model_GNNNet_kiba.model'.")

            # Load the pretrained model weights and adjust the keys
            state_dict = torch.load('model_GNNNet_kiba.model', map_location=device)
            adjusted_state_dict = adjust_state_dict(state_dict, model.state_dict())
            model.load_state_dict(adjusted_state_dict, strict=False)

            # The optimizer remains as initialized

        # Training loop with progress bar over epochs
        for epoch in tqdm(range(start_epoch, num_epochs + 1),
                          desc=f"Training Fold {fold_number}", unit="epoch"):
            model.train()
            running_loss = 0.0

            # Prepare batch loader without progress bar for batches
            batch_size = 256 # Adjust batch size as needed
            batch_loader_iter = batch_loader(train_files, sample_dir, batch_size=batch_size)

            for batch_samples in batch_loader_iter:
                mol_data_list = []
                pro_data_list = []
                target_list = []

                for sample in batch_samples:
                    mol_data = sample[0]
                    pro_data = sample[1]
                    target = sample[2]

                    mol_data_list.append(mol_data)
                    pro_data_list.append(pro_data)
                    target_list.append(target)

                mol_batch = Batch.from_data_list(mol_data_list).to(device)
                pro_batch = Batch.from_data_list(pro_data_list).to(device)
                target = torch.tensor(target_list, dtype=torch.float32).view(-1).to(device)

                optimizer.zero_grad()
                output = model(mol_batch, pro_batch)
                loss = loss_fn(output.view(-1), target)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * len(batch_samples)

            avg_loss = running_loss / len(train_files)
            # Use tqdm.write() to print without interfering with the progress bar
            tqdm.write(f"Fold {fold_number}, Epoch {epoch}/{num_epochs}, Loss: {avg_loss:.4f}")

            # Save the model and optimizer states after each epoch
            checkpoint_filename = f"model_fold{fold_number}_epoch{epoch}.pt"
            checkpoint_path = os.path.join(training_model_dir, checkpoint_filename)
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, checkpoint_path)
            tqdm.write(f"Checkpoint saved for fold {fold_number} at epoch {epoch}")

            # Evaluation on the test set after each epoch
            model.eval()
            total_preds, total_labels = [], []
            with torch.no_grad():
                batch_size = 256  # Adjust batch size as needed
                batch_loader_iter = batch_loader(test_files, sample_dir, batch_size=batch_size)

                for batch_samples in batch_loader_iter:
                    mol_data_list = []
                    pro_data_list = []
                    target_list = []

                    for sample in batch_samples:
                        mol_data = sample[0]
                        pro_data = sample[1]
                        target = sample[2]

                        mol_data_list.append(mol_data)
                        pro_data_list.append(pro_data)
                        target_list.append(target)

                    mol_batch = Batch.from_data_list(mol_data_list).to(device)
                    pro_batch = Batch.from_data_list(pro_data_list).to(device)
                    target = torch.tensor(target_list, dtype=torch.float32).view(-1).to(device)

                    output = model(mol_batch, pro_batch)
                    total_preds.append(output.cpu().numpy())
                    total_labels.append(target.cpu().numpy())

            # Convert lists to numpy arrays for evaluation
            total_preds = np.concatenate(total_preds)
            total_labels = np.concatenate(total_labels)

            # Calculate metrics
            mse = get_mse(total_labels, total_preds)
            ci = get_ci(total_labels, total_preds)
            pearson = get_pearson(total_labels, total_preds)

            # Print metrics
            tqdm.write(f"Fold {fold_number}, Epoch {epoch}/{num_epochs} - MSE: {mse:.4f}, CI: {ci:.4f}, Pearson: {pearson:.4f}")

        # Evaluation at the end of training for this fold
        print(f"Final evaluation for Fold {fold_number}: MSE: {mse:.4f}, CI: {ci:.4f}, Pearson: {pearson:.4f}")
        # Store results for this fold
        results.append((mse, ci, pearson))

    return results


if __name__ == "__main__":
    sample_dir = 'prepared_samples'  # Adjust the path to your samples directory
    num_epochs = 5  # Adjust the number of epochs as needed
    n_splits = 5  # Number of folds for cross-validation
    learning_rate = 0.001  # Learning rate

    # Run the training function
    results = train_5fold_cross_validation(sample_dir, num_epochs=num_epochs, n_splits=n_splits, lr=learning_rate)

    # Print overall results
    print("\nCross-validation Results:")
    for fold_idx, (mse, ci, pearson) in enumerate(results):
        print(f"Fold {fold_idx + 1}: MSE={mse:.4f}, CI={ci:.4f}, Pearson={pearson:.4f}")

    # Optionally, compute and print average metrics across folds
    mse_values, ci_values, pearson_values = zip(*results)
    print(f"\nAverage Results:")
    print(f"MSE: {np.mean(mse_values):.4f}")
    print(f"CI: {np.mean(ci_values):.4f}")
    print(f"Pearson Correlation: {np.mean(pearson_values):.4f}")


Running on cpu.
Using existing TrainingModel directory at prepared_samples/TrainingModel

Fold 1/5
GNNNet Loaded
Model is on device: cpu
No checkpoint found for fold 1, loading pretrained model from 'model_GNNNet_kiba.model'.


Training Fold 1:   0%|          | 0/5 [15:42<?, ?epoch/s]

Fold 1, Epoch 1/5, Loss: 10.3111
Checkpoint saved for fold 1 at epoch 1


Training Fold 1:  20%|██        | 1/5 [23:32<1:34:09, 1412.46s/epoch]

Fold 1, Epoch 1/5 - MSE: 0.6774, CI: 0.4904, Pearson: -0.0099


...

Fold 1, Epoch 4/5, Loss: 0.6910
Checkpoint saved for fold 1 at epoch 4


Training Fold 1:  80%|████████  | 4/5 [1:37:16<24:27, 1467.79s/epoch]

Fold 1, Epoch 4/5 - MSE: 0.6788, CI: 0.5495, Pearson: 0.1465


Training Fold 1:  80%|████████  | 4/5 [1:48:29<27:07, 1627.32s/epoch]


KeyboardInterrupt: 

In [18]:
import os
import torch
import torch.nn as nn
import numpy as np
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv, global_mean_pool as gep
from tqdm import tqdm
from scipy.stats import pearsonr, spearmanr

# Define metric functions
def get_mse(y_true, y_pred):
    return np.mean((y_pred - y_true) ** 2)

def get_ci(y_true, y_pred):
    n = 0
    h_sum = 0.0
    for i in range(len(y_true)):
        for j in range(len(y_true)):
            if y_true[i] > y_true[j]:
                n += 1
                if y_pred[i] > y_pred[j]:
                    h_sum += 1
                elif y_pred[i] == y_pred[j]:
                    h_sum += 0.5
    return h_sum / n if n != 0 else 0

def get_pearson(y_true, y_pred):
    return pearsonr(y_true, y_pred)[0]

def get_spearman(y_true, y_pred):
    return spearmanr(y_true, y_pred)[0]

def get_rmse(y_true, y_pred):
    return np.sqrt(get_mse(y_true, y_pred))

def get_rm2(y_true, y_pred):
    y_mean = np.mean(y_true)
    ss_tot = np.sum((y_true - y_mean) ** 2)
    ss_res = np.sum((y_true - y_pred) ** 2)
    r_squared = 1 - (ss_res / ss_tot)
    return r_squared

import torch

def adjust_state_dict(state_dict):
    new_state_dict = {}
    for k in state_dict.keys():
        new_k = k
        # Adjust mol_conv layers
        if 'mol_conv' in k:
            if 'lin.weight' in k:
                new_k = k.replace('lin.weight', 'weight')
            elif 'lin.bias' in k:
                new_k = k.replace('lin.bias', 'bias')
        # Adjust pro_conv layers
        elif 'pro_conv' in k:
            if 'lin.weight' in k:
                new_k = k.replace('lin.weight', 'weight')
            elif 'lin.bias' in k:
                new_k = k.replace('lin.bias', 'bias')
        new_state_dict[new_k] = state_dict[k]
    return new_state_dict




# Define the load_sample function
def load_sample(path):
    data = torch.load(path)
    if isinstance(data, tuple) and len(data) == 3:
        mol_data, pro_data, target = data
    elif isinstance(data, dict):
        mol_data = data.get('mol_data')
        pro_data = data.get('pro_data')
        target = data.get('target')
    else:
        print(f"Unexpected data format in file {path}")
        return None

    # Convert dictionaries to Data objects if necessary
    if isinstance(mol_data, dict):
        mol_data = Data(**mol_data)
    if isinstance(pro_data, dict):
        pro_data = Data(**pro_data)

    # Ensure that 'batch' attribute is set
    if not hasattr(mol_data, 'batch') or mol_data.batch is None:
        mol_data.batch = torch.zeros(mol_data.num_nodes, dtype=torch.long)
    if not hasattr(pro_data, 'batch') or pro_data.batch is None:
        pro_data.batch = torch.zeros(pro_data.num_nodes, dtype=torch.long)

    return (mol_data, pro_data, target)

# Define the predicting function
def predicting(model, device, samples):
    model.eval()
    total_preds = []
    total_labels = []
    print('Making predictions for {} samples...'.format(len(samples)))
    with torch.no_grad():
        batch_size = 256  # Adjust as necessary
        for i in tqdm(range(0, len(samples), batch_size), desc='Predicting'):
            batch_samples = samples[i:i+batch_size]
            mol_data_list = []
            pro_data_list = []
            target_list = []
            for sample in batch_samples:
                if sample is None:
                    continue
                mol_data, pro_data, target = sample
                mol_data_list.append(mol_data)
                pro_data_list.append(pro_data)
                target_list.append(target)
            if len(mol_data_list) == 0:
                continue
            mol_batch = Batch.from_data_list(mol_data_list).to(device)
            pro_batch = Batch.from_data_list(pro_data_list).to(device)
            targets = torch.tensor(target_list, dtype=torch.float32).view(-1).to(device)
            outputs = model(mol_batch, pro_batch)
            total_preds.append(outputs.cpu().numpy())
            total_labels.append(targets.cpu().numpy())
    if len(total_preds) == 0:
        print("No predictions were made.")
        return None, None
    total_preds = np.concatenate(total_preds)
    total_labels = np.concatenate(total_labels)
    return total_labels, total_preds

# Define the main function
def main():
    # Set the sample directory where your data is stored
    sample_dir = 'prepared_samples'  # Adjust the path to your available samples
    model_path = 'model_GNNNet_kiba.model'  # Path to your pretrained model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Using device:', device)
    
    # Load the model
    # Determine input feature dimensions from your data
    sample_files = [f for f in os.listdir(sample_dir) if f.endswith('.pt')]
    if not sample_files:
        print(f"No sample files found in {sample_dir}")
        return
    sample_file_path = os.path.join(sample_dir, sample_files[0])
    sample = load_sample(sample_file_path)
    if sample is None:
        print("Failed to load sample.")
        return

    mol_data, pro_data, _ = sample

    num_features_mol = mol_data.x.size(1)
    num_features_pro = pro_data.x.size(1)
    
    model = GNNNet(num_features_mol=num_features_mol, num_features_pro=num_features_pro).to(device)
    
    
    # Load the pretrained model weights and adjust the keys
    state_dict = torch.load(model_path, map_location=device)
    adjusted_state_dict = adjust_state_dict(state_dict, model.state_dict())
    model.load_state_dict(adjusted_state_dict, strict=False)
    
    # Load the data
    samples = []
    for sample_file in sample_files:
        sample_path = os.path.join(sample_dir, sample_file)
        sample = load_sample(sample_path)
        if sample is not None:
            samples.append(sample)
    if not samples:
        print(f"No valid samples found in {sample_dir}")
        return
    
    # Run predictions
    Y, P = predicting(model, device, samples)
    if Y is None or P is None:
        print("No predictions were made.")
        return
    
    # Calculate metrics
    mse = get_mse(Y, P)
    ci = get_ci(Y, P)
    pearson = get_pearson(Y, P)
    spearman = get_spearman(Y, P)
    rmse = get_rmse(Y, P)
    rm2 = get_rm2(Y, P)
    
    # Print metrics
    print('Evaluation Metrics:')
    print(f'MSE: {mse:.4f}')
    print(f'RMSE: {rmse:.4f}')
    print(f'Pearson Correlation: {pearson:.4f}')
    print(f'Spearman Correlation: {spearman:.4f}')
    print(f'CI: {ci:.4f}')
    print(f'R-squared (RM2): {rm2:.4f}')
    
    # Optionally, plot the results
    # plot_density(Y, P)

if __name__ == '__main__':
    main()


Using device: cpu


TypeError: zeros() received an invalid combination of arguments - got (NoneType, dtype=torch.dtype), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [5]:
# import os
# import torch
# import torch.optim as optim
# from torch.nn import MSELoss
# from sklearn.model_selection import KFold
# import numpy as np
# from tqdm import tqdm
# from torch_geometric.data import Data, Batch
# from torch_geometric.nn import GCNConv, global_mean_pool as gep
# from scipy.stats import pearsonr
# import warnings
# import itertools
# from collections import OrderedDict

# # Suppress FutureWarning related to torch.load
# warnings.filterwarnings('ignore', category=FutureWarning)

# # Define the load_sample function
# def load_sample(path):
#     # Load individual sample from file
#     sample = torch.load(path)
#     mol_data = sample[0]
#     pro_data = sample[1]
#     target = sample[2]

#     # Convert dictionaries to Data objects if necessary
#     if isinstance(mol_data, dict):
#         mol_data = Data(**mol_data)
#     if isinstance(pro_data, dict):
#         pro_data = Data(**pro_data)

#     # Ensure that 'x' attribute is set
#     if not hasattr(mol_data, 'x') or mol_data.x is None:
#         if hasattr(mol_data, 'features'):
#             mol_data.x = mol_data.features
#             del mol_data.features
#         else:
#             raise ValueError("mol_data does not have 'x' or 'features' attribute")

#     if not hasattr(pro_data, 'x') or pro_data.x is None:
#         if hasattr(pro_data, 'features'):
#             pro_data.x = pro_data.features
#             del pro_data.features
#         else:
#             raise ValueError("pro_data does not have 'x' or 'features' attribute")

#     # Ensure 'x' is a float tensor
#     if not isinstance(mol_data.x, torch.Tensor):
#         mol_data.x = torch.tensor(mol_data.x)
#     if not isinstance(pro_data.x, torch.Tensor):
#         pro_data.x = torch.tensor(pro_data.x)

#     if mol_data.x.dtype != torch.float:
#         mol_data.x = mol_data.x.float()
#     if pro_data.x.dtype != torch.float:
#         pro_data.x = pro_data.x.float()

#     # Adjust 'edge_index' for mol_data
#     if not isinstance(mol_data.edge_index, torch.Tensor):
#         mol_data.edge_index = torch.tensor(mol_data.edge_index, dtype=torch.long)
#     else:
#         mol_data.edge_index = mol_data.edge_index.long()

#     if mol_data.edge_index.shape[0] != 2:
#         mol_data.edge_index = mol_data.edge_index.t()

#     # Adjust 'edge_index' for pro_data
#     if not isinstance(pro_data.edge_index, torch.Tensor):
#         pro_data.edge_index = torch.tensor(pro_data.edge_index, dtype=torch.long)
#     else:
#         pro_data.edge_index = pro_data.edge_index.long()

#     if pro_data.edge_index.shape[0] != 2:
#         pro_data.edge_index = pro_data.edge_index.t()

#     # Set 'num_nodes' attribute to suppress warnings
#     mol_data.num_nodes = mol_data.x.size(0)
#     pro_data.num_nodes = pro_data.x.size(0)

#     return (mol_data, pro_data, target)

# # Define the batch_loader function
# def batch_loader(file_list, sample_dir, batch_size):
#     batch = []
#     for idx, file_name in enumerate(file_list):
#         sample_path = os.path.join(sample_dir, file_name)
#         sample = load_sample(sample_path)
#         batch.append(sample)
#         if len(batch) == batch_size:
#             yield batch
#             batch = []
#     if len(batch) > 0:
#         yield batch

# # Define the evaluation metrics functions
# def get_mse(y_true, y_pred):
#     return np.mean((y_true - y_pred) ** 2)

# def get_ci(y_true, y_pred):
#     """
#     Compute the concordance index between true and predicted values.
#     """
#     pairs = itertools.combinations(range(len(y_true)), 2)
#     c = 0
#     s = 0
#     for i, j in pairs:
#         if y_true[i] != y_true[j]:
#             s += 1
#             if (y_true[i] < y_true[j] and y_pred[i] < y_pred[j]) or \
#                (y_true[i] > y_true[j] and y_pred[i] > y_pred[j]):
#                 c += 1
#             elif y_pred[i] == y_pred[j]:
#                 c += 0.5
#     return c / s if s != 0 else 0

# def get_pearson(y_true, y_pred):
#     return pearsonr(y_true.flatten(), y_pred.flatten())[0]

# def train_5fold_cross_validation(sample_dir, num_epochs=1000, n_splits=5, lr=0.001):
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     print(f"Running on {device}.")

#     sample_files = [f for f in os.listdir(sample_dir) if f.endswith('.pt')]
#     kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

#     # Create a single directory for all checkpoints
#     training_model_dir = os.path.join(sample_dir, 'TrainingModel')
#     if not os.path.exists(training_model_dir):
#         os.makedirs(training_model_dir)
#         print(f"Created directory for checkpoints at {training_model_dir}")
#     else:
#         print(f"Using existing TrainingModel directory at {training_model_dir}")

#     results = []
#     loss_fn = MSELoss()

#     for fold, (train_idx, test_idx) in enumerate(kfold.split(sample_files)):
#         fold_number = fold + 1
#         print(f'\nFold {fold_number}/{n_splits}')
#         train_files = [sample_files[i] for i in train_idx]
#         test_files = [sample_files[i] for i in test_idx]

#         # Determine input feature dimensions from your data
#         sample = load_sample(os.path.join(sample_dir, train_files[0]))
#         mol_data = sample[0]
#         pro_data = sample[1]

#         num_features_mol = 78
#         num_features_pro = 54

#         # Initialize the GNN model with correct input dimensions
#         model = GNNNet(
#             num_features_mol=num_features_mol,
#             num_features_pro=num_features_pro
#         ).to(device)
#         print(f"Model is on device: {next(model.parameters()).device}")

#         # Initialize starting epoch
#         start_epoch = 1

#         # Check for existing checkpoints in TrainingModel directory for the current fold
#         existing_checkpoints = [f for f in os.listdir(training_model_dir)
#                                 if f.endswith('.pt') and f.startswith(f'model_fold{fold_number}_epoch')]

#         if existing_checkpoints:
#             # Find the latest checkpoint based on epoch number
#             latest_checkpoint = max(existing_checkpoints,
#                                     key=lambda x: int(x.split('_epoch')[1].split('.pt')[0]))
#             checkpoint_path = os.path.join(training_model_dir, latest_checkpoint)
#             print(f"Loading checkpoint for fold {fold_number} from {checkpoint_path}")
#             checkpoint = torch.load(checkpoint_path, map_location=device)

#             # Determine the format of the checkpoint and load accordingly
#             if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
#                 # Standard checkpoint format
#                 model.load_state_dict(checkpoint['model_state_dict'])
#                 optimizer = optim.Adam(model.parameters(), lr=lr)
#                 optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#                 loaded_epoch = checkpoint['epoch']
#                 start_epoch = loaded_epoch + 1
#                 print(f"Resuming training from epoch {start_epoch}")
#             else:
#                 # Checkpoint is the model's state_dict directly
#                 model.load_state_dict(checkpoint)
#                 optimizer = optim.Adam(model.parameters(), lr=lr)
#                 # Since epoch is not saved, start from 1 or set as needed
#                 start_epoch = 1
#                 print("Loaded model state_dict directly from checkpoint.")
#                 print(f"Starting training from epoch {start_epoch}")
#         else:
#             print(f"No checkpoint found for fold {fold_number}, starting training from scratch.")
#             # Load pre-trained model weights
#             model_weight_path = 'model_GNNNet_kiba.model'  # Adjust the path accordingly
#             if os.path.exists(model_weight_path):
#                 # Load the state_dict from the file
#                 state_dict = torch.load(model_weight_path, map_location=device)
#                 # Load the state_dict into the model
#                 if isinstance(state_dict, dict) and 'model_state_dict' in state_dict:
#                     # If state_dict has 'model_state_dict' key, use it
#                     model.load_state_dict(state_dict['model_state_dict'])
#                     print(f"Loaded pre-trained model weights from {model_weight_path} using 'model_state_dict' key.")
#                 else:
#                     # Assume state_dict is the model's state_dict directly
#                     model.load_state_dict(state_dict)
#                     print(f"Loaded pre-trained model weights from {model_weight_path}")
#             else:
#                 print(f"Pre-trained model weights not found at {model_weight_path}, proceeding without loading weights.")
#             optimizer = optim.Adam(model.parameters(), lr=lr)
#             start_epoch = 1

#         # Training loop with progress bar over epochs
#         for epoch in tqdm(range(start_epoch, num_epochs + 1),
#                           desc=f"Training Fold {fold_number}", unit="epoch"):
#             model.train()
#             running_loss = 0.0

#             # Prepare batch loader without progress bar for batches
#             batch_size = 128  # Adjust batch size as needed
#             batch_loader_iter = batch_loader(train_files, sample_dir, batch_size=batch_size)

#             for batch_samples in batch_loader_iter:
#                 mol_data_list = []
#                 pro_data_list = []
#                 target_list = []

#                 for sample in batch_samples:
#                     mol_data = sample[0]
#                     pro_data = sample[1]
#                     target = sample[2]

#                     mol_data_list.append(mol_data)
#                     pro_data_list.append(pro_data)
#                     target_list.append(target)

#                 mol_batch = Batch.from_data_list(mol_data_list).to(device)
#                 pro_batch = Batch.from_data_list(pro_data_list).to(device)
#                 target = torch.tensor(target_list, dtype=torch.float32).view(-1).to(device)

#                 optimizer.zero_grad()
#                 output = model(mol_batch, pro_batch)
#                 loss = loss_fn(output.view(-1), target)
#                 loss.backward()
#                 optimizer.step()
#                 running_loss += loss.item() * len(batch_samples)

#             avg_loss = running_loss / len(train_files)
#             # Use tqdm.write() to print without interfering with the progress bar
#             tqdm.write(f"Fold {fold_number}, Epoch {epoch}/{num_epochs}, Loss: {avg_loss:.4f}")

#             # Save the model and optimizer states after each epoch
#             checkpoint_filename = f"model_fold{fold_number}_epoch{epoch}.pt"
#             checkpoint_path = os.path.join(training_model_dir, checkpoint_filename)
#             torch.save({
#                 'epoch': epoch,
#                 'model_state_dict': model.state_dict(),
#                 'optimizer_state_dict': optimizer.state_dict(),
#             }, checkpoint_path)
#             tqdm.write(f"Checkpoint saved for fold {fold_number} at epoch {epoch}")

#             # Evaluation on the test set after each epoch
#             model.eval()
#             total_preds, total_labels = [], []
#             with torch.no_grad():
#                 batch_size = 128  # Adjust batch size as needed
#                 batch_loader_iter = batch_loader(test_files, sample_dir, batch_size=batch_size)

#                 for batch_samples in batch_loader_iter:
#                     mol_data_list = []
#                     pro_data_list = []
#                     target_list = []

#                     for sample in batch_samples:
#                         mol_data = sample[0]
#                         pro_data = sample[1]
#                         target = sample[2]

#                         mol_data_list.append(mol_data)
#                         pro_data_list.append(pro_data)
#                         target_list.append(target)

#                     mol_batch = Batch.from_data_list(mol_data_list).to(device)
#                     pro_batch = Batch.from_data_list(pro_data_list).to(device)
#                     target = torch.tensor(target_list, dtype=torch.float32).view(-1).to(device)

#                     output = model(mol_batch, pro_batch)
#                     total_preds.append(output.cpu().numpy())
#                     total_labels.append(target.cpu().numpy())

#             # Convert lists to numpy arrays for evaluation
#             total_preds = np.concatenate(total_preds)
#             total_labels = np.concatenate(total_labels)

#             # Calculate metrics
#             mse = get_mse(total_labels, total_preds)
#             ci = get_ci(total_labels, total_preds)
#             pearson = get_pearson(total_labels, total_preds)

#             # Print metrics
#             tqdm.write(f"Fold {fold_number}, Epoch {epoch}/{num_epochs} - MSE: {mse:.4f}, CI: {ci:.4f}, Pearson: {pearson:.4f}")

#         # Evaluation at the end of training for this fold
#         print(f"Final evaluation for Fold {fold_number}: MSE: {mse:.4f}, CI: {ci:.4f}, Pearson: {pearson:.4f}")
#         # Store results for this fold
#         results.append((mse, ci, pearson))

#     return results

# # Example usage
# if __name__ == "__main__":
#     sample_dir = 'prepared_samples'  # Adjust the path to your samples directory
#     num_epochs = 5  # Adjust the number of epochs as needed
#     n_splits = 5  # Number of folds for cross-validation
#     learning_rate = 0.001  # Learning rate

#     # Run the training function
#     results = train_5fold_cross_validation(sample_dir, num_epochs=num_epochs, n_splits=n_splits, lr=learning_rate)

#     # Print overall results
#     print("\nCross-validation Results:")
#     for fold_idx, (mse, ci, pearson) in enumerate(results):
#         print(f"Fold {fold_idx + 1}: MSE={mse:.4f}, CI={ci:.4f}, Pearson={pearson:.4f}")

#     # Optionally, compute and print average metrics across folds
#     mse_values, ci_values, pearson_values = zip(*results)
#     print(f"\nAverage Results:")
#     print(f"MSE: {np.mean(mse_values):.4f}")
#     print(f"CI: {np.mean(ci_values):.4f}")
#     print(f"Pearson Correlation: {np.mean(pearson_values):.4f}")


In [6]:
import os
import torch
import torch.optim as optim
from torch.nn import MSELoss
from sklearn.model_selection import KFold
import numpy as np
from tqdm import tqdm
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv, global_mean_pool as gep
from scipy.stats import pearsonr
import warnings
import itertools

# Suppress FutureWarning related to torch.load
warnings.filterwarnings('ignore', category=FutureWarning)


# Define the load_sample function
def load_sample(path):
    # Load individual sample from file
    sample = torch.load(path)
    mol_data = sample[0]
    pro_data = sample[1]
    target = sample[2]

    # Convert dictionaries to Data objects if necessary
    if isinstance(mol_data, dict):
        mol_data = Data(**mol_data)
    if isinstance(pro_data, dict):
        pro_data = Data(**pro_data)

    # Ensure that 'x' attribute is set
    if not hasattr(mol_data, 'x') or mol_data.x is None:
        if hasattr(mol_data, 'features'):
            mol_data.x = mol_data.features
            del mol_data.features
        else:
            raise ValueError("mol_data does not have 'x' or 'features' attribute")

    if not hasattr(pro_data, 'x') or pro_data.x is None:
        if hasattr(pro_data, 'features'):
            pro_data.x = pro_data.features
            del pro_data.features
        else:
            raise ValueError("pro_data does not have 'x' or 'features' attribute")

    # Ensure 'x' is a float tensor
    if not isinstance(mol_data.x, torch.Tensor):
        mol_data.x = torch.tensor(mol_data.x)
    if not isinstance(pro_data.x, torch.Tensor):
        pro_data.x = torch.tensor(pro_data.x)

    if mol_data.x.dtype != torch.float:
        mol_data.x = mol_data.x.float()
    if pro_data.x.dtype != torch.float:
        pro_data.x = pro_data.x.float()

    # Adjust 'edge_index' for mol_data
    # Ensure 'edge_index' is a tensor of type torch.long
    if not isinstance(mol_data.edge_index, torch.Tensor):
        mol_data.edge_index = torch.tensor(mol_data.edge_index, dtype=torch.long)
    else:
        mol_data.edge_index = mol_data.edge_index.long()

    # Ensure 'edge_index' has shape [2, num_edges]
    if mol_data.edge_index.shape[0] != 2:
        mol_data.edge_index = mol_data.edge_index.t()

    # Adjust 'edge_index' for pro_data
    if not isinstance(pro_data.edge_index, torch.Tensor):
        pro_data.edge_index = torch.tensor(pro_data.edge_index, dtype=torch.long)
    else:
        pro_data.edge_index = pro_data.edge_index.long()

    if pro_data.edge_index.shape[0] != 2:
        pro_data.edge_index = pro_data.edge_index.t()

    # Set 'num_nodes' attribute to suppress warnings
    mol_data.num_nodes = mol_data.x.size(0)
    pro_data.num_nodes = pro_data.x.size(0)

    return (mol_data, pro_data, target)

# Define the batch_loader function
def batch_loader(file_list, sample_dir, batch_size):
    batch = []
    for idx, file_name in enumerate(file_list):
        sample_path = os.path.join(sample_dir, file_name)
        sample = load_sample(sample_path)
        batch.append(sample)
        if len(batch) == batch_size:
            yield batch
            batch = []
    if len(batch) > 0:
        yield batch

# Define the evaluation metrics functions
def get_mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def get_ci(y_true, y_pred):
    """
    Compute the concordance index between true and predicted values.
    """
    pairs = itertools.combinations(range(len(y_true)), 2)
    c = 0
    s = 0
    for i, j in pairs:
        if y_true[i] != y_true[j]:
            s += 1
            if (y_true[i] < y_true[j] and y_pred[i] < y_pred[j]) or \
               (y_true[i] > y_true[j] and y_pred[i] > y_pred[j]):
                c += 1
            elif y_pred[i] == y_pred[j]:
                c += 0.5
    return c / s if s != 0 else 0

def get_pearson(y_true, y_pred):
    return pearsonr(y_true.flatten(), y_pred.flatten())[0]

def train_5fold_cross_validation(sample_dir, num_epochs=1000, n_splits=5, lr=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Running on {device}.")

    sample_files = [f for f in os.listdir(sample_dir) if f.endswith('.pt')]
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Create a single directory for all checkpoints
    training_model_dir = os.path.join(sample_dir, 'TrainingModel')
    if not os.path.exists(training_model_dir):
        os.makedirs(training_model_dir)
        print(f"Created directory for checkpoints at {training_model_dir}")
    else:
        print(f"Using existing TrainingModel directory at {training_model_dir}")

    results = []
    loss_fn = MSELoss()

    for fold, (train_idx, test_idx) in enumerate(kfold.split(sample_files)):
        fold_number = fold + 1
        print(f'\nFold {fold_number}/{n_splits}')
        train_files = [sample_files[i] for i in train_idx]
        test_files = [sample_files[i] for i in test_idx]

        # Determine input feature dimensions from your data
        sample = load_sample(os.path.join(sample_dir, train_files[0]))
        mol_data = sample[0]
        pro_data = sample[1]

        num_features_mol = mol_data.x.size(1)
        num_features_pro = pro_data.x.size(1)

        # Initialize the GNN model with correct input dimensions
        model = GNNNet(
            num_features_mol=num_features_mol,
            num_features_pro=num_features_pro
        ).to(device)
        print(f"Model is on device: {next(model.parameters()).device}")

        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Initialize starting epoch
        start_epoch = 1

        # Check for existing checkpoints in TrainingModel directory for the current fold
        existing_checkpoints = [f for f in os.listdir(training_model_dir)
                                if f.endswith('.pt') and f.startswith(f'model_fold{fold_number}_epoch')]

        if existing_checkpoints:
            # Find the latest checkpoint based on epoch number
            latest_checkpoint = max(existing_checkpoints,
                                    key=lambda x: int(x.split('_epoch')[1].split('.pt')[0]))
            checkpoint_path = os.path.join(training_model_dir, latest_checkpoint)
            print(f"Loading checkpoint for fold {fold_number} from {checkpoint_path}")
            checkpoint = torch.load(checkpoint_path, map_location=device)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            loaded_epoch = checkpoint['epoch']
            start_epoch = loaded_epoch + 1
            print(f"Resuming training from epoch {start_epoch}")
        else:
            print(f"No checkpoint found for fold {fold_number}, starting training from scratch.")

        # Training loop with progress bar over epochs
        for epoch in tqdm(range(start_epoch, num_epochs + 1),
                          desc=f"Training Fold {fold_number}", unit="epoch"):
            model.train()
            running_loss = 0.0

            # Prepare batch loader without progress bar for batches
            batch_size = 128  # Adjust batch size as needed
            batch_loader_iter = batch_loader(train_files, sample_dir, batch_size=batch_size)

            for batch_samples in batch_loader_iter:
                mol_data_list = []
                pro_data_list = []
                target_list = []

                for sample in batch_samples:
                    mol_data = sample[0]
                    pro_data = sample[1]
                    target = sample[2]

                    mol_data_list.append(mol_data)
                    pro_data_list.append(pro_data)
                    target_list.append(target)

                mol_batch = Batch.from_data_list(mol_data_list).to(device)
                pro_batch = Batch.from_data_list(pro_data_list).to(device)
                target = torch.tensor(target_list, dtype=torch.float32).view(-1).to(device)

                optimizer.zero_grad()
                output = model(mol_batch, pro_batch)
                loss = loss_fn(output.view(-1), target)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * len(batch_samples)

            avg_loss = running_loss / len(train_files)
            # Use tqdm.write() to print without interfering with the progress bar
            tqdm.write(f"Fold {fold_number}, Epoch {epoch}/{num_epochs}, Loss: {avg_loss:.4f}")

            # Save the model and optimizer states after each epoch
            checkpoint_filename = f"model_fold{fold_number}_epoch{epoch}.pt"
            checkpoint_path = os.path.join(training_model_dir, checkpoint_filename)
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, checkpoint_path)
            tqdm.write(f"Checkpoint saved for fold {fold_number} at epoch {epoch}")

            # Evaluation on the test set after each epoch
            model.eval()
            total_preds, total_labels = [], []
            with torch.no_grad():
                batch_size = 128  # Adjust batch size as needed
                batch_loader_iter = batch_loader(test_files, sample_dir, batch_size=batch_size)

                for batch_samples in batch_loader_iter:
                    mol_data_list = []
                    pro_data_list = []
                    target_list = []

                    for sample in batch_samples:
                        mol_data = sample[0]
                        pro_data = sample[1]
                        target = sample[2]

                        mol_data_list.append(mol_data)
                        pro_data_list.append(pro_data)
                        target_list.append(target)

                    mol_batch = Batch.from_data_list(mol_data_list).to(device)
                    pro_batch = Batch.from_data_list(pro_data_list).to(device)
                    target = torch.tensor(target_list, dtype=torch.float32).view(-1).to(device)

                    output = model(mol_batch, pro_batch)
                    total_preds.append(output.cpu().numpy())
                    total_labels.append(target.cpu().numpy())

            # Convert lists to numpy arrays for evaluation
            total_preds = np.concatenate(total_preds)
            total_labels = np.concatenate(total_labels)

            # Calculate metrics
            mse = get_mse(total_labels, total_preds)
            ci = get_ci(total_labels, total_preds)
            pearson = get_pearson(total_labels, total_preds)

            # Print metrics
            tqdm.write(f"Fold {fold_number}, Epoch {epoch}/{num_epochs} - MSE: {mse:.4f}, CI: {ci:.4f}, Pearson: {pearson:.4f}")

        # Evaluation at the end of training for this fold
        print(f"Final evaluation for Fold {fold_number}: MSE: {mse:.4f}, CI: {ci:.4f}, Pearson: {pearson:.4f}")
        # Store results for this fold
        results.append((mse, ci, pearson))

    return results

# Example usage
if __name__ == "__main__":
    sample_dir = 'prepared_samples'  # Adjust the path to your samples directory
    num_epochs = 100  # Adjust the number of epochs as needed
    n_splits = 5  # Number of folds for cross-validation
    learning_rate = 0.001  # Learning rate

    # Run the training function
    results = train_5fold_cross_validation(sample_dir, num_epochs=num_epochs, n_splits=n_splits, lr=learning_rate)

    # Print overall results
    print("\nCross-validation Results:")
    for fold_idx, (mse, ci, pearson) in enumerate(results):
        print(f"Fold {fold_idx + 1}: MSE={mse:.4f}, CI={ci:.4f}, Pearson={pearson:.4f}")

    # Optionally, compute and print average metrics across folds
    mse_values, ci_values, pearson_values = zip(*results)
    print(f"\nAverage Results:")
    print(f"MSE: {np.mean(mse_values):.4f}")
    print(f"CI: {np.mean(ci_values):.4f}")
    print(f"Pearson Correlation: {np.mean(pearson_values):.4f}")


Running on cpu.
Using existing TrainingModel directory at prepared_samples/TrainingModel

Fold 1/5
GNNNet Loaded
Model is on device: cpu
Loading checkpoint for fold 1 from prepared_samples/TrainingModel/model_fold1_epoch2.pt
Resuming training from epoch 3


Training Fold 1:   0%|          | 0/98 [11:41<?, ?epoch/s]

Fold 1, Epoch 3/100, Loss: 1.1182
Checkpoint saved for fold 1 at epoch 3


Training Fold 1:   1%|          | 1/98 [18:24<29:45:14, 1104.28s/epoch]

Fold 1, Epoch 3/100 - MSE: 1.2433, CI: 0.6028, Pearson: 0.3476


...

Fold 1, Epoch 30/100, Loss: 0.4785
Checkpoint saved for fold 1 at epoch 30


Training Fold 1:  29%|██▊       | 28/98 [8:34:36<21:12:01, 1090.30s/epoch]

Fold 1, Epoch 30/100 - MSE: 1.0663, CI: 0.7858, Pearson: 0.7343


Training Fold 1:  29%|██▊       | 28/98 [8:51:49<22:09:33, 1139.63s/epoch]


KeyboardInterrupt: 

In [7]:
import os
import torch
import torch.optim as optim
from torch.nn import MSELoss
from sklearn.model_selection import KFold
import numpy as np
from tqdm import tqdm
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv, global_mean_pool as gep
from scipy.stats import pearsonr
import warnings
import itertools

# Suppress FutureWarning related to torch.load
warnings.filterwarnings('ignore', category=FutureWarning)


# Define the load_sample function
def load_sample(path):
    # Load individual sample from file
    sample = torch.load(path)
    mol_data = sample[0]
    pro_data = sample[1]
    target = sample[2]

    # Convert dictionaries to Data objects if necessary
    if isinstance(mol_data, dict):
        mol_data = Data(**mol_data)
    if isinstance(pro_data, dict):
        pro_data = Data(**pro_data)

    # Ensure that 'x' attribute is set
    if not hasattr(mol_data, 'x') or mol_data.x is None:
        if hasattr(mol_data, 'features'):
            mol_data.x = mol_data.features
            del mol_data.features
        else:
            raise ValueError("mol_data does not have 'x' or 'features' attribute")

    if not hasattr(pro_data, 'x') or pro_data.x is None:
        if hasattr(pro_data, 'features'):
            pro_data.x = pro_data.features
            del pro_data.features
        else:
            raise ValueError("pro_data does not have 'x' or 'features' attribute")

    # Ensure 'x' is a float tensor
    if not isinstance(mol_data.x, torch.Tensor):
        mol_data.x = torch.tensor(mol_data.x)
    if not isinstance(pro_data.x, torch.Tensor):
        pro_data.x = torch.tensor(pro_data.x)

    if mol_data.x.dtype != torch.float:
        mol_data.x = mol_data.x.float()
    if pro_data.x.dtype != torch.float:
        pro_data.x = pro_data.x.float()

    # Adjust 'edge_index' for mol_data
    # Ensure 'edge_index' is a tensor of type torch.long
    if not isinstance(mol_data.edge_index, torch.Tensor):
        mol_data.edge_index = torch.tensor(mol_data.edge_index, dtype=torch.long)
    else:
        mol_data.edge_index = mol_data.edge_index.long()

    # Ensure 'edge_index' has shape [2, num_edges]
    if mol_data.edge_index.shape[0] != 2:
        mol_data.edge_index = mol_data.edge_index.t()

    # Adjust 'edge_index' for pro_data
    if not isinstance(pro_data.edge_index, torch.Tensor):
        pro_data.edge_index = torch.tensor(pro_data.edge_index, dtype=torch.long)
    else:
        pro_data.edge_index = pro_data.edge_index.long()

    if pro_data.edge_index.shape[0] != 2:
        pro_data.edge_index = pro_data.edge_index.t()

    # Set 'num_nodes' attribute to suppress warnings
    mol_data.num_nodes = mol_data.x.size(0)
    pro_data.num_nodes = pro_data.x.size(0)

    return (mol_data, pro_data, target)

# Define the batch_loader function
def batch_loader(file_list, sample_dir, batch_size):
    batch = []
    for idx, file_name in enumerate(file_list):
        sample_path = os.path.join(sample_dir, file_name)
        sample = load_sample(sample_path)
        batch.append(sample)
        if len(batch) == batch_size:
            yield batch
            batch = []
    if len(batch) > 0:
        yield batch

# Define the evaluation metrics functions
def get_mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def get_ci(y_true, y_pred):
    """
    Compute the concordance index between true and predicted values.
    """
    pairs = itertools.combinations(range(len(y_true)), 2)
    c = 0
    s = 0
    for i, j in pairs:
        if y_true[i] != y_true[j]:
            s += 1
            if (y_true[i] < y_true[j] and y_pred[i] < y_pred[j]) or \
               (y_true[i] > y_true[j] and y_pred[i] > y_pred[j]):
                c += 1
            elif y_pred[i] == y_pred[j]:
                c += 0.5
    return c / s if s != 0 else 0

def get_pearson(y_true, y_pred):
    return pearsonr(y_true.flatten(), y_pred.flatten())[0]

def train_5fold_cross_validation(sample_dir, num_epochs=1000, n_splits=5, lr=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Running on {device}.")

    sample_files = [f for f in os.listdir(sample_dir) if f.endswith('.pt')]
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Create a single directory for all checkpoints
    training_model_dir = os.path.join(sample_dir, 'TrainingModel')
    if not os.path.exists(training_model_dir):
        os.makedirs(training_model_dir)
        print(f"Created directory for checkpoints at {training_model_dir}")
    else:
        print(f"Using existing TrainingModel directory at {training_model_dir}")

    results = []
    loss_fn = MSELoss()

    for fold, (train_idx, test_idx) in enumerate(kfold.split(sample_files)):
        fold_number = fold + 1
        print(f'\nFold {fold_number}/{n_splits}')
        train_files = [sample_files[i] for i in train_idx]
        test_files = [sample_files[i] for i in test_idx]

        # Determine input feature dimensions from your data
        sample = load_sample(os.path.join(sample_dir, train_files[0]))
        mol_data = sample[0]
        pro_data = sample[1]

        num_features_mol = mol_data.x.size(1)
        num_features_pro = pro_data.x.size(1)

        # Initialize the GNN model with correct input dimensions
        model = GNNNet(
            num_features_mol=num_features_mol,
            num_features_pro=num_features_pro
        ).to(device)
        print(f"Model is on device: {next(model.parameters()).device}")

        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Initialize starting epoch
        start_epoch = 1

        # Check for existing checkpoints in TrainingModel directory for the current fold
        existing_checkpoints = [f for f in os.listdir(training_model_dir)
                                if f.endswith('.pt') and f.startswith(f'model_fold{fold_number}_epoch')]

        if existing_checkpoints:
            # Find the latest checkpoint based on epoch number
            latest_checkpoint = max(existing_checkpoints,
                                    key=lambda x: int(x.split('_epoch')[1].split('.pt')[0]))
            checkpoint_path = os.path.join(training_model_dir, latest_checkpoint)
            print(f"Loading checkpoint for fold {fold_number} from {checkpoint_path}")
            checkpoint = torch.load(checkpoint_path, map_location=device)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            loaded_epoch = checkpoint['epoch']
            start_epoch = loaded_epoch + 1
            print(f"Resuming training from epoch {start_epoch}")
        else:
            print(f"No checkpoint found for fold {fold_number}, starting training from scratch.")

        # Training loop with progress bar over epochs
        for epoch in tqdm(range(start_epoch, num_epochs + 1),
                          desc=f"Training Fold {fold_number}", unit="epoch"):
            model.train()
            running_loss = 0.0

            # Prepare batch loader without progress bar for batches
            batch_size = 200  # Adjust batch size as needed
            batch_loader_iter = batch_loader(train_files, sample_dir, batch_size=batch_size)

            for batch_samples in batch_loader_iter:
                mol_data_list = []
                pro_data_list = []
                target_list = []

                for sample in batch_samples:
                    mol_data = sample[0]
                    pro_data = sample[1]
                    target = sample[2]

                    mol_data_list.append(mol_data)
                    pro_data_list.append(pro_data)
                    target_list.append(target)

                mol_batch = Batch.from_data_list(mol_data_list).to(device)
                pro_batch = Batch.from_data_list(pro_data_list).to(device)
                target = torch.tensor(target_list, dtype=torch.float32).view(-1).to(device)

                optimizer.zero_grad()
                output = model(mol_batch, pro_batch)
                loss = loss_fn(output.view(-1), target)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * len(batch_samples)

            avg_loss = running_loss / len(train_files)
            # Use tqdm.write() to print without interfering with the progress bar
            tqdm.write(f"Fold {fold_number}, Epoch {epoch}/{num_epochs}, Loss: {avg_loss:.4f}")

            # Save the model and optimizer states after each epoch
            checkpoint_filename = f"model_fold{fold_number}_epoch{epoch}.pt"
            checkpoint_path = os.path.join(training_model_dir, checkpoint_filename)
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, checkpoint_path)
            tqdm.write(f"Checkpoint saved for fold {fold_number} at epoch {epoch}")

            # Evaluation on the test set after each epoch
            model.eval()
            total_preds, total_labels = [], []
            with torch.no_grad():
                batch_size = 200  # Adjust batch size as needed
                batch_loader_iter = batch_loader(test_files, sample_dir, batch_size=batch_size)

                for batch_samples in batch_loader_iter:
                    mol_data_list = []
                    pro_data_list = []
                    target_list = []

                    for sample in batch_samples:
                        mol_data = sample[0]
                        pro_data = sample[1]
                        target = sample[2]

                        mol_data_list.append(mol_data)
                        pro_data_list.append(pro_data)
                        target_list.append(target)

                    mol_batch = Batch.from_data_list(mol_data_list).to(device)
                    pro_batch = Batch.from_data_list(pro_data_list).to(device)
                    target = torch.tensor(target_list, dtype=torch.float32).view(-1).to(device)

                    output = model(mol_batch, pro_batch)
                    total_preds.append(output.cpu().numpy())
                    total_labels.append(target.cpu().numpy())

            # Convert lists to numpy arrays for evaluation
            total_preds = np.concatenate(total_preds)
            total_labels = np.concatenate(total_labels)

            # Calculate metrics
            mse = get_mse(total_labels, total_preds)
            ci = get_ci(total_labels, total_preds)
            pearson = get_pearson(total_labels, total_preds)

            # Print metrics
            tqdm.write(f"Fold {fold_number}, Epoch {epoch}/{num_epochs} - MSE: {mse:.4f}, CI: {ci:.4f}, Pearson: {pearson:.4f}")

        # Evaluation at the end of training for this fold
        print(f"Final evaluation for Fold {fold_number}: MSE: {mse:.4f}, CI: {ci:.4f}, Pearson: {pearson:.4f}")
        # Store results for this fold
        results.append((mse, ci, pearson))

    return results


if __name__ == "__main__":
    sample_dir = 'prepared_samples'  # Adjust the path to your samples directory
    num_epochs = 120  # Adjust the number of epochs as needed
    n_splits = 5  # Number of folds for cross-validation
    learning_rate = 0.001  # Learning rate

    # Run the training function
    results = train_5fold_cross_validation(sample_dir, num_epochs=num_epochs, n_splits=n_splits, lr=learning_rate)

    # Print overall results
    print("\nCross-validation Results:")
    for fold_idx, (mse, ci, pearson) in enumerate(results):
        print(f"Fold {fold_idx + 1}: MSE={mse:.4f}, CI={ci:.4f}, Pearson={pearson:.4f}")

    # Optionally, compute and print average metrics across folds
    mse_values, ci_values, pearson_values = zip(*results)
    print(f"\nAverage Results:")
    print(f"MSE: {np.mean(mse_values):.4f}")
    print(f"CI: {np.mean(ci_values):.4f}")
    print(f"Pearson Correlation: {np.mean(pearson_values):.4f}")


Running on cpu.
Using existing TrainingModel directory at prepared_samples/TrainingModel

Fold 1/5
GNNNet Loaded
Model is on device: cpu
Loading checkpoint for fold 1 from prepared_samples/TrainingModel/model_fold1_epoch30.pt
Resuming training from epoch 31


Training Fold 1:   0%|          | 0/90 [11:58<?, ?epoch/s]

Fold 1, Epoch 31/120, Loss: 0.4399
Checkpoint saved for fold 1 at epoch 31


Training Fold 1:   1%|          | 1/90 [18:14<27:04:14, 1095.00s/epoch]

Fold 1, Epoch 31/120 - MSE: 1.0760, CI: 0.7888, Pearson: 0.7425


...

Fold 1, Epoch 51/120 - MSE: 1.0294, CI: 0.8115, Pearson: 0.7814


Training Fold 1:  23%|██▎       | 21/90 [6:50:43<22:21:48, 1166.79s/epoch]

Fold 1, Epoch 52/120, Loss: 0.3633
Checkpoint saved for fold 1 at epoch 52


Training Fold 1:  23%|██▎       | 21/90 [6:54:49<22:42:59, 1185.21s/epoch]


KeyboardInterrupt: 

In [8]:
import os
import torch
import torch.optim as optim
from torch.nn import MSELoss
from sklearn.model_selection import KFold
import numpy as np
from tqdm import tqdm
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv, global_mean_pool as gep
from scipy.stats import pearsonr
import warnings
import itertools

# Suppress FutureWarning related to torch.load
warnings.filterwarnings('ignore', category=FutureWarning)


# Define the load_sample function
def load_sample(path):
    # Load individual sample from file
    sample = torch.load(path)
    mol_data = sample[0]
    pro_data = sample[1]
    target = sample[2]

    # Convert dictionaries to Data objects if necessary
    if isinstance(mol_data, dict):
        mol_data = Data(**mol_data)
    if isinstance(pro_data, dict):
        pro_data = Data(**pro_data)

    # Ensure that 'x' attribute is set
    if not hasattr(mol_data, 'x') or mol_data.x is None:
        if hasattr(mol_data, 'features'):
            mol_data.x = mol_data.features
            del mol_data.features
        else:
            raise ValueError("mol_data does not have 'x' or 'features' attribute")

    if not hasattr(pro_data, 'x') or pro_data.x is None:
        if hasattr(pro_data, 'features'):
            pro_data.x = pro_data.features
            del pro_data.features
        else:
            raise ValueError("pro_data does not have 'x' or 'features' attribute")

    # Ensure 'x' is a float tensor
    if not isinstance(mol_data.x, torch.Tensor):
        mol_data.x = torch.tensor(mol_data.x)
    if not isinstance(pro_data.x, torch.Tensor):
        pro_data.x = torch.tensor(pro_data.x)

    if mol_data.x.dtype != torch.float:
        mol_data.x = mol_data.x.float()
    if pro_data.x.dtype != torch.float:
        pro_data.x = pro_data.x.float()

    # Adjust 'edge_index' for mol_data
    # Ensure 'edge_index' is a tensor of type torch.long
    if not isinstance(mol_data.edge_index, torch.Tensor):
        mol_data.edge_index = torch.tensor(mol_data.edge_index, dtype=torch.long)
    else:
        mol_data.edge_index = mol_data.edge_index.long()

    # Ensure 'edge_index' has shape [2, num_edges]
    if mol_data.edge_index.shape[0] != 2:
        mol_data.edge_index = mol_data.edge_index.t()

    # Adjust 'edge_index' for pro_data
    if not isinstance(pro_data.edge_index, torch.Tensor):
        pro_data.edge_index = torch.tensor(pro_data.edge_index, dtype=torch.long)
    else:
        pro_data.edge_index = pro_data.edge_index.long()

    if pro_data.edge_index.shape[0] != 2:
        pro_data.edge_index = pro_data.edge_index.t()

    # Set 'num_nodes' attribute to suppress warnings
    mol_data.num_nodes = mol_data.x.size(0)
    pro_data.num_nodes = pro_data.x.size(0)

    return (mol_data, pro_data, target)

# Define the batch_loader function
def batch_loader(file_list, sample_dir, batch_size):
    batch = []
    for idx, file_name in enumerate(file_list):
        sample_path = os.path.join(sample_dir, file_name)
        sample = load_sample(sample_path)
        batch.append(sample)
        if len(batch) == batch_size:
            yield batch
            batch = []
    if len(batch) > 0:
        yield batch

# Define the evaluation metrics functions
def get_mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def get_ci(y_true, y_pred):
    """
    Compute the concordance index between true and predicted values.
    """
    pairs = itertools.combinations(range(len(y_true)), 2)
    c = 0
    s = 0
    for i, j in pairs:
        if y_true[i] != y_true[j]:
            s += 1
            if (y_true[i] < y_true[j] and y_pred[i] < y_pred[j]) or \
               (y_true[i] > y_true[j] and y_pred[i] > y_pred[j]):
                c += 1
            elif y_pred[i] == y_pred[j]:
                c += 0.5
    return c / s if s != 0 else 0

def get_pearson(y_true, y_pred):
    return pearsonr(y_true.flatten(), y_pred.flatten())[0]

def train_5fold_cross_validation(sample_dir, num_epochs=1000, n_splits=5, lr=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Running on {device}.")

    sample_files = [f for f in os.listdir(sample_dir) if f.endswith('.pt')]
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Create a single directory for all checkpoints
    training_model_dir = os.path.join(sample_dir, 'TrainingModel')
    if not os.path.exists(training_model_dir):
        os.makedirs(training_model_dir)
        print(f"Created directory for checkpoints at {training_model_dir}")
    else:
        print(f"Using existing TrainingModel directory at {training_model_dir}")

    results = []
    loss_fn = MSELoss()

    for fold, (train_idx, test_idx) in enumerate(kfold.split(sample_files)):
        fold_number = fold + 1
        print(f'\nFold {fold_number}/{n_splits}')
        train_files = [sample_files[i] for i in train_idx]
        test_files = [sample_files[i] for i in test_idx]

        # Determine input feature dimensions from your data
        sample = load_sample(os.path.join(sample_dir, train_files[0]))
        mol_data = sample[0]
        pro_data = sample[1]

        num_features_mol = mol_data.x.size(1)
        num_features_pro = pro_data.x.size(1)

        # Initialize the GNN model with correct input dimensions
        model = GNNNet(
            num_features_mol=num_features_mol,
            num_features_pro=num_features_pro
        ).to(device)
        print(f"Model is on device: {next(model.parameters()).device}")

        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Initialize starting epoch
        start_epoch = 1

        # Check for existing checkpoints in TrainingModel directory for the current fold
        existing_checkpoints = [f for f in os.listdir(training_model_dir)
                                if f.endswith('.pt') and f.startswith(f'model_fold{fold_number}_epoch')]

        if existing_checkpoints:
            # Find the latest checkpoint based on epoch number
            latest_checkpoint = max(existing_checkpoints,
                                    key=lambda x: int(x.split('_epoch')[1].split('.pt')[0]))
            checkpoint_path = os.path.join(training_model_dir, latest_checkpoint)
            print(f"Loading checkpoint for fold {fold_number} from {checkpoint_path}")
            checkpoint = torch.load(checkpoint_path, map_location=device)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            loaded_epoch = checkpoint['epoch']
            start_epoch = loaded_epoch + 1
            print(f"Resuming training from epoch {start_epoch}")
        else:
            print(f"No checkpoint found for fold {fold_number}, starting training from scratch.")

        # Training loop with progress bar over epochs
        for epoch in tqdm(range(start_epoch, num_epochs + 1),
                          desc=f"Training Fold {fold_number}", unit="epoch"):
            model.train()
            running_loss = 0.0

            # Prepare batch loader without progress bar for batches
            batch_size = 256 # Adjust batch size as needed
            batch_loader_iter = batch_loader(train_files, sample_dir, batch_size=batch_size)

            for batch_samples in batch_loader_iter:
                mol_data_list = []
                pro_data_list = []
                target_list = []

                for sample in batch_samples:
                    mol_data = sample[0]
                    pro_data = sample[1]
                    target = sample[2]

                    mol_data_list.append(mol_data)
                    pro_data_list.append(pro_data)
                    target_list.append(target)

                mol_batch = Batch.from_data_list(mol_data_list).to(device)
                pro_batch = Batch.from_data_list(pro_data_list).to(device)
                target = torch.tensor(target_list, dtype=torch.float32).view(-1).to(device)

                optimizer.zero_grad()
                output = model(mol_batch, pro_batch)
                loss = loss_fn(output.view(-1), target)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * len(batch_samples)

            avg_loss = running_loss / len(train_files)
            # Use tqdm.write() to print without interfering with the progress bar
            tqdm.write(f"Fold {fold_number}, Epoch {epoch}/{num_epochs}, Loss: {avg_loss:.4f}")

            # Save the model and optimizer states after each epoch
            checkpoint_filename = f"model_fold{fold_number}_epoch{epoch}.pt"
            checkpoint_path = os.path.join(training_model_dir, checkpoint_filename)
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, checkpoint_path)
            tqdm.write(f"Checkpoint saved for fold {fold_number} at epoch {epoch}")

            # Evaluation on the test set after each epoch
            model.eval()
            total_preds, total_labels = [], []
            with torch.no_grad():
                batch_size = 256  # Adjust batch size as needed
                batch_loader_iter = batch_loader(test_files, sample_dir, batch_size=batch_size)

                for batch_samples in batch_loader_iter:
                    mol_data_list = []
                    pro_data_list = []
                    target_list = []

                    for sample in batch_samples:
                        mol_data = sample[0]
                        pro_data = sample[1]
                        target = sample[2]

                        mol_data_list.append(mol_data)
                        pro_data_list.append(pro_data)
                        target_list.append(target)

                    mol_batch = Batch.from_data_list(mol_data_list).to(device)
                    pro_batch = Batch.from_data_list(pro_data_list).to(device)
                    target = torch.tensor(target_list, dtype=torch.float32).view(-1).to(device)

                    output = model(mol_batch, pro_batch)
                    total_preds.append(output.cpu().numpy())
                    total_labels.append(target.cpu().numpy())

            # Convert lists to numpy arrays for evaluation
            total_preds = np.concatenate(total_preds)
            total_labels = np.concatenate(total_labels)

            # Calculate metrics
            mse = get_mse(total_labels, total_preds)
            ci = get_ci(total_labels, total_preds)
            pearson = get_pearson(total_labels, total_preds)

            # Print metrics
            tqdm.write(f"Fold {fold_number}, Epoch {epoch}/{num_epochs} - MSE: {mse:.4f}, CI: {ci:.4f}, Pearson: {pearson:.4f}")

        # Evaluation at the end of training for this fold
        print(f"Final evaluation for Fold {fold_number}: MSE: {mse:.4f}, CI: {ci:.4f}, Pearson: {pearson:.4f}")
        # Store results for this fold
        results.append((mse, ci, pearson))

    return results


if __name__ == "__main__":
    sample_dir = 'prepared_samples'  # Adjust the path to your samples directory
    num_epochs = 250  # Adjust the number of epochs as needed
    n_splits = 5  # Number of folds for cross-validation
    learning_rate = 0.001  # Learning rate

    # Run the training function
    results = train_5fold_cross_validation(sample_dir, num_epochs=num_epochs, n_splits=n_splits, lr=learning_rate)

    # Print overall results
    print("\nCross-validation Results:")
    for fold_idx, (mse, ci, pearson) in enumerate(results):
        print(f"Fold {fold_idx + 1}: MSE={mse:.4f}, CI={ci:.4f}, Pearson={pearson:.4f}")

    # Optionally, compute and print average metrics across folds
    mse_values, ci_values, pearson_values = zip(*results)
    print(f"\nAverage Results:")
    print(f"MSE: {np.mean(mse_values):.4f}")
    print(f"CI: {np.mean(ci_values):.4f}")
    print(f"Pearson Correlation: {np.mean(pearson_values):.4f}")


Running on cpu.
Using existing TrainingModel directory at prepared_samples/TrainingModel

Fold 1/5
GNNNet Loaded
Model is on device: cpu
Loading checkpoint for fold 1 from prepared_samples/TrainingModel/model_fold1_epoch52.pt
Resuming training from epoch 53


Training Fold 1:   0%|          | 0/198 [13:06<?, ?epoch/s]

Fold 1, Epoch 53/250, Loss: 0.3572
Checkpoint saved for fold 1 at epoch 53


Training Fold 1:   1%|          | 1/198 [19:33<64:12:53, 1173.47s/epoch]

Fold 1, Epoch 53/250 - MSE: 1.0370, CI: 0.8181, Pearson: 0.7939


...

Fold 1, Epoch 133/250 - MSE: 1.1685, CI: 0.8558, Pearson: 0.8508


Training Fold 1:  41%|████      | 81/198 [25:59:49<37:38:13, 1158.06s/epoch]

Fold 1, Epoch 134/250, Loss: 0.2096
Checkpoint saved for fold 1 at epoch 134


Training Fold 1:  41%|████      | 81/198 [26:03:10<37:37:54, 1157.90s/epoch]


KeyboardInterrupt: 

In [9]:
import os
import torch
import torch.optim as optim
from torch.nn import MSELoss
from sklearn.model_selection import KFold
import numpy as np
from tqdm import tqdm
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv, global_mean_pool as gep
from scipy.stats import pearsonr
import warnings
import itertools

# Suppress FutureWarning related to torch.load
warnings.filterwarnings('ignore', category=FutureWarning)


# Define the load_sample function
def load_sample(path):
    # Load individual sample from file
    sample = torch.load(path)
    mol_data = sample[0]
    pro_data = sample[1]
    target = sample[2]

    # Convert dictionaries to Data objects if necessary
    if isinstance(mol_data, dict):
        mol_data = Data(**mol_data)
    if isinstance(pro_data, dict):
        pro_data = Data(**pro_data)

    # Ensure that 'x' attribute is set
    if not hasattr(mol_data, 'x') or mol_data.x is None:
        if hasattr(mol_data, 'features'):
            mol_data.x = mol_data.features
            del mol_data.features
        else:
            raise ValueError("mol_data does not have 'x' or 'features' attribute")

    if not hasattr(pro_data, 'x') or pro_data.x is None:
        if hasattr(pro_data, 'features'):
            pro_data.x = pro_data.features
            del pro_data.features
        else:
            raise ValueError("pro_data does not have 'x' or 'features' attribute")

    # Ensure 'x' is a float tensor
    if not isinstance(mol_data.x, torch.Tensor):
        mol_data.x = torch.tensor(mol_data.x)
    if not isinstance(pro_data.x, torch.Tensor):
        pro_data.x = torch.tensor(pro_data.x)

    if mol_data.x.dtype != torch.float:
        mol_data.x = mol_data.x.float()
    if pro_data.x.dtype != torch.float:
        pro_data.x = pro_data.x.float()

    # Adjust 'edge_index' for mol_data
    # Ensure 'edge_index' is a tensor of type torch.long
    if not isinstance(mol_data.edge_index, torch.Tensor):
        mol_data.edge_index = torch.tensor(mol_data.edge_index, dtype=torch.long)
    else:
        mol_data.edge_index = mol_data.edge_index.long()

    # Ensure 'edge_index' has shape [2, num_edges]
    if mol_data.edge_index.shape[0] != 2:
        mol_data.edge_index = mol_data.edge_index.t()

    # Adjust 'edge_index' for pro_data
    if not isinstance(pro_data.edge_index, torch.Tensor):
        pro_data.edge_index = torch.tensor(pro_data.edge_index, dtype=torch.long)
    else:
        pro_data.edge_index = pro_data.edge_index.long()

    if pro_data.edge_index.shape[0] != 2:
        pro_data.edge_index = pro_data.edge_index.t()

    # Set 'num_nodes' attribute to suppress warnings
    mol_data.num_nodes = mol_data.x.size(0)
    pro_data.num_nodes = pro_data.x.size(0)

    return (mol_data, pro_data, target)

# Define the batch_loader function
def batch_loader(file_list, sample_dir, batch_size):
    batch = []
    for idx, file_name in enumerate(file_list):
        sample_path = os.path.join(sample_dir, file_name)
        sample = load_sample(sample_path)
        batch.append(sample)
        if len(batch) == batch_size:
            yield batch
            batch = []
    if len(batch) > 0:
        yield batch

# Define the evaluation metrics functions
def get_mse(y_true, y_pred):
    return np.mean((y_pred - y_true ) ** 2)

def get_ci(y_true, y_pred):
    """
    Compute the concordance index between true and predicted values.
    """
    pairs = itertools.combinations(range(len(y_true)), 2)
    c = 0
    s = 0
    for i, j in pairs:
        if y_true[i] != y_true[j]:
            s += 1
            if (y_true[i] < y_true[j] and y_pred[i] < y_pred[j]) or \
               (y_true[i] > y_true[j] and y_pred[i] > y_pred[j]):
                c += 1
            elif y_pred[i] == y_pred[j]:
                c += 0.5
    return c / s if s != 0 else 0

def get_pearson(y_true, y_pred):
    return pearsonr(y_true.flatten(), y_pred.flatten())[0]

def train_5fold_cross_validation(sample_dir, num_epochs=1000, n_splits=5, lr=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Running on {device}.")

    sample_files = [f for f in os.listdir(sample_dir) if f.endswith('.pt')]
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Create a single directory for all checkpoints
    training_model_dir = os.path.join(sample_dir, 'TrainingModel')
    if not os.path.exists(training_model_dir):
        os.makedirs(training_model_dir)
        print(f"Created directory for checkpoints at {training_model_dir}")
    else:
        print(f"Using existing TrainingModel directory at {training_model_dir}")

    results = []
    loss_fn = MSELoss()

    for fold, (train_idx, test_idx) in enumerate(kfold.split(sample_files)):
        fold_number = fold + 1
        print(f'\nFold {fold_number}/{n_splits}')
        train_files = [sample_files[i] for i in train_idx]
        test_files = [sample_files[i] for i in test_idx]

        # Determine input feature dimensions from your data
        sample = load_sample(os.path.join(sample_dir, train_files[0]))
        mol_data = sample[0]
        pro_data = sample[1]

        num_features_mol = mol_data.x.size(1)
        num_features_pro = pro_data.x.size(1)

        # Initialize the GNN model with correct input dimensions
        model = GNNNet(
            num_features_mol=num_features_mol,
            num_features_pro=num_features_pro
        ).to(device)
        print(f"Model is on device: {next(model.parameters()).device}")

        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Initialize starting epoch
        start_epoch = 1

        # Check for existing checkpoints in TrainingModel directory for the current fold
        existing_checkpoints = [f for f in os.listdir(training_model_dir)
                                if f.endswith('.pt') and f.startswith(f'model_fold{fold_number}_epoch')]

        if existing_checkpoints:
            # Find the latest checkpoint based on epoch number
            latest_checkpoint = max(existing_checkpoints,
                                    key=lambda x: int(x.split('_epoch')[1].split('.pt')[0]))
            checkpoint_path = os.path.join(training_model_dir, latest_checkpoint)
            print(f"Loading checkpoint for fold {fold_number} from {checkpoint_path}")
            checkpoint = torch.load(checkpoint_path, map_location=device)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            loaded_epoch = checkpoint['epoch']
            start_epoch = loaded_epoch + 1
            print(f"Resuming training from epoch {start_epoch}")
        else:
            print(f"No checkpoint found for fold {fold_number}, starting training from scratch.")

        # Training loop with progress bar over epochs
        for epoch in tqdm(range(start_epoch, num_epochs + 1),
                          desc=f"Training Fold {fold_number}", unit="epoch"):
            model.train()
            running_loss = 0.0

            # Prepare batch loader without progress bar for batches
            batch_size = 256 # Adjust batch size as needed
            batch_loader_iter = batch_loader(train_files, sample_dir, batch_size=batch_size)

            for batch_samples in batch_loader_iter:
                mol_data_list = []
                pro_data_list = []
                target_list = []

                for sample in batch_samples:
                    mol_data = sample[0]
                    pro_data = sample[1]
                    target = sample[2]

                    mol_data_list.append(mol_data)
                    pro_data_list.append(pro_data)
                    target_list.append(target)

                mol_batch = Batch.from_data_list(mol_data_list).to(device)
                pro_batch = Batch.from_data_list(pro_data_list).to(device)
                target = torch.tensor(target_list, dtype=torch.float32).view(-1).to(device)

                optimizer.zero_grad()
                output = model(mol_batch, pro_batch)
                loss = loss_fn(output.view(-1), target)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * len(batch_samples)

            avg_loss = running_loss / len(train_files)
            # Use tqdm.write() to print without interfering with the progress bar
            tqdm.write(f"Fold {fold_number}, Epoch {epoch}/{num_epochs}, Loss: {avg_loss:.4f}")

            # Save the model and optimizer states after each epoch
            checkpoint_filename = f"model_fold{fold_number}_epoch{epoch}.pt"
            checkpoint_path = os.path.join(training_model_dir, checkpoint_filename)
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, checkpoint_path)
            tqdm.write(f"Checkpoint saved for fold {fold_number} at epoch {epoch}")

            # Evaluation on the test set after each epoch
            model.eval()
            total_preds, total_labels = [], []
            with torch.no_grad():
                batch_size = 256  # Adjust batch size as needed
                batch_loader_iter = batch_loader(test_files, sample_dir, batch_size=batch_size)

                for batch_samples in batch_loader_iter:
                    mol_data_list = []
                    pro_data_list = []
                    target_list = []

                    for sample in batch_samples:
                        mol_data = sample[0]
                        pro_data = sample[1]
                        target = sample[2]

                        mol_data_list.append(mol_data)
                        pro_data_list.append(pro_data)
                        target_list.append(target)

                    mol_batch = Batch.from_data_list(mol_data_list).to(device)
                    pro_batch = Batch.from_data_list(pro_data_list).to(device)
                    target = torch.tensor(target_list, dtype=torch.float32).view(-1).to(device)

                    output = model(mol_batch, pro_batch)
                    total_preds.append(output.cpu().numpy())
                    total_labels.append(target.cpu().numpy())

            # Convert lists to numpy arrays for evaluation
            total_preds = np.concatenate(total_preds)
            total_labels = np.concatenate(total_labels)

            # Calculate metrics
            mse = get_mse(total_labels, total_preds)
            ci = get_ci(total_labels, total_preds)
            pearson = get_pearson(total_labels, total_preds)

            # Print metrics
            tqdm.write(f"Fold {fold_number}, Epoch {epoch}/{num_epochs} - MSE: {mse:.4f}, CI: {ci:.4f}, Pearson: {pearson:.4f}")

        # Evaluation at the end of training for this fold
        print(f"Final evaluation for Fold {fold_number}: MSE: {mse:.4f}, CI: {ci:.4f}, Pearson: {pearson:.4f}")
        # Store results for this fold
        results.append((mse, ci, pearson))

    return results


if __name__ == "__main__":
    sample_dir = 'prepared_samples'  # Adjust the path to your samples directory
    num_epochs = 250  # Adjust the number of epochs as needed
    n_splits = 5  # Number of folds for cross-validation
    learning_rate = 0.001  # Learning rate

    # Run the training function
    results = train_5fold_cross_validation(sample_dir, num_epochs=num_epochs, n_splits=n_splits, lr=learning_rate)

    # Print overall results
    print("\nCross-validation Results:")
    for fold_idx, (mse, ci, pearson) in enumerate(results):
        print(f"Fold {fold_idx + 1}: MSE={mse:.4f}, CI={ci:.4f}, Pearson={pearson:.4f}")

    # Optionally, compute and print average metrics across folds
    mse_values, ci_values, pearson_values = zip(*results)
    print(f"\nAverage Results:")
    print(f"MSE: {np.mean(mse_values):.4f}")
    print(f"CI: {np.mean(ci_values):.4f}")
    print(f"Pearson Correlation: {np.mean(pearson_values):.4f}")


Running on cpu.
Using existing TrainingModel directory at prepared_samples/TrainingModel

Fold 1/5
GNNNet Loaded
Model is on device: cpu
Loading checkpoint for fold 1 from prepared_samples/TrainingModel/model_fold1_epoch134.pt
Resuming training from epoch 135


Training Fold 1:   0%|          | 0/116 [13:33<?, ?epoch/s]

Fold 1, Epoch 135/250, Loss: 0.2120
Checkpoint saved for fold 1 at epoch 135


Training Fold 1:   1%|          | 1/116 [20:02<38:24:04, 1202.13s/epoch]

Fold 1, Epoch 135/250 - MSE: 1.1602, CI: 0.8533, Pearson: 0.8513


...

Fold 1, Epoch 174/250, Loss: 0.1788
Checkpoint saved for fold 1 at epoch 174


Training Fold 1:  34%|███▍      | 40/116 [12:50:11<24:01:42, 1138.19s/epoch]

Fold 1, Epoch 174/250 - MSE: 1.1817, CI: 0.8635, Pearson: 0.8620


Training Fold 1:  34%|███▍      | 40/116 [12:58:53<24:39:53, 1168.34s/epoch]


KeyboardInterrupt: 

In [12]:
import os
import torch
import torch.optim as optim
from torch.nn import MSELoss
from sklearn.model_selection import KFold
import numpy as np
from tqdm import tqdm
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv, global_mean_pool as gep
from scipy.stats import pearsonr
import warnings
import itertools

# Suppress FutureWarning related to torch.load
warnings.filterwarnings('ignore', category=FutureWarning)


# Define the load_sample function
def load_sample(path):
    # Load individual sample from file
    sample = torch.load(path)
    mol_data = sample[0]
    pro_data = sample[1]
    target = sample[2]

    # Convert dictionaries to Data objects if necessary
    if isinstance(mol_data, dict):
        mol_data = Data(**mol_data)
    if isinstance(pro_data, dict):
        pro_data = Data(**pro_data)

    # Ensure that 'x' attribute is set
    if not hasattr(mol_data, 'x') or mol_data.x is None:
        if hasattr(mol_data, 'features'):
            mol_data.x = mol_data.features
            del mol_data.features
        else:
            raise ValueError("mol_data does not have 'x' or 'features' attribute")

    if not hasattr(pro_data, 'x') or pro_data.x is None:
        if hasattr(pro_data, 'features'):
            pro_data.x = pro_data.features
            del pro_data.features
        else:
            raise ValueError("pro_data does not have 'x' or 'features' attribute")

    # Ensure 'x' is a float tensor
    if not isinstance(mol_data.x, torch.Tensor):
        mol_data.x = torch.tensor(mol_data.x)
    if not isinstance(pro_data.x, torch.Tensor):
        pro_data.x = torch.tensor(pro_data.x)

    if mol_data.x.dtype != torch.float:
        mol_data.x = mol_data.x.float()
    if pro_data.x.dtype != torch.float:
        pro_data.x = pro_data.x.float()

    # Adjust 'edge_index' for mol_data
    # Ensure 'edge_index' is a tensor of type torch.long
    if not isinstance(mol_data.edge_index, torch.Tensor):
        mol_data.edge_index = torch.tensor(mol_data.edge_index, dtype=torch.long)
    else:
        mol_data.edge_index = mol_data.edge_index.long()

    # Ensure 'edge_index' has shape [2, num_edges]
    if mol_data.edge_index.shape[0] != 2:
        mol_data.edge_index = mol_data.edge_index.t()

    # Adjust 'edge_index' for pro_data
    if not isinstance(pro_data.edge_index, torch.Tensor):
        pro_data.edge_index = torch.tensor(pro_data.edge_index, dtype=torch.long)
    else:
        pro_data.edge_index = pro_data.edge_index.long()

    if pro_data.edge_index.shape[0] != 2:
        pro_data.edge_index = pro_data.edge_index.t()

    # Set 'num_nodes' attribute to suppress warnings
    mol_data.num_nodes = mol_data.x.size(0)
    pro_data.num_nodes = pro_data.x.size(0)

    return (mol_data, pro_data, target)

# Define the batch_loader function
def batch_loader(file_list, sample_dir, batch_size):
    batch = []
    for idx, file_name in enumerate(file_list):
        sample_path = os.path.join(sample_dir, file_name)
        sample = load_sample(sample_path)
        batch.append(sample)
        if len(batch) == batch_size:
            yield batch
            batch = []
    if len(batch) > 0:
        yield batch

# Define the evaluation metrics functions
def get_mse(y_true, y_pred):
    return np.mean((y_pred - y_true ) ** 2)

def get_ci(y_true, y_pred):
    """
    Compute the concordance index between true and predicted values.
    """
    pairs = itertools.combinations(range(len(y_true)), 2)
    c = 0
    s = 0
    for i, j in pairs:
        if y_true[i] != y_true[j]:
            s += 1
            if (y_true[i] < y_true[j] and y_pred[i] < y_pred[j]) or \
               (y_true[i] > y_true[j] and y_pred[i] > y_pred[j]):
                c += 1
            elif y_pred[i] == y_pred[j]:
                c += 0.5
    return c / s if s != 0 else 0

def get_pearson(y_true, y_pred):
    return pearsonr(y_true.flatten(), y_pred.flatten())[0]

def train_5fold_cross_validation(sample_dir, num_epochs=1000, n_splits=5, lr=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Running on {device}.")

    sample_files = [f for f in os.listdir(sample_dir) if f.endswith('.pt')]
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Create a single directory for all checkpoints
    training_model_dir = os.path.join(sample_dir, 'TrainingModel')
    if not os.path.exists(training_model_dir):
        os.makedirs(training_model_dir)
        print(f"Created directory for checkpoints at {training_model_dir}")
    else:
        print(f"Using existing TrainingModel directory at {training_model_dir}")

    results = []
    loss_fn = MSELoss()

    for fold, (train_idx, test_idx) in enumerate(kfold.split(sample_files)):
        fold_number = fold + 1
        print(f'\nFold {fold_number}/{n_splits}')
        train_files = [sample_files[i] for i in train_idx]
        test_files = [sample_files[i] for i in test_idx]

        # Determine input feature dimensions from your data
        sample = load_sample(os.path.join(sample_dir, train_files[0]))
        mol_data = sample[0]
        pro_data = sample[1]

        num_features_mol = mol_data.x.size(1)
        num_features_pro = pro_data.x.size(1)

        # Initialize the GNN model with correct input dimensions
        model = GNNNet(
            num_features_mol=num_features_mol,
            num_features_pro=num_features_pro
        ).to(device)
        print(f"Model is on device: {next(model.parameters()).device}")

        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Initialize starting epoch
        start_epoch = 1

        # Check for existing checkpoints in TrainingModel directory for the current fold
        existing_checkpoints = [f for f in os.listdir(training_model_dir)
                                if f.endswith('.pt') and f.startswith(f'model_fold{fold_number}_epoch')]

        if existing_checkpoints:
            # Find the latest checkpoint based on epoch number
            latest_checkpoint = max(existing_checkpoints,
                                    key=lambda x: int(x.split('_epoch')[1].split('.pt')[0]))
            checkpoint_path = os.path.join(training_model_dir, latest_checkpoint)
            print(f"Loading checkpoint for fold {fold_number} from {checkpoint_path}")
            checkpoint = torch.load(checkpoint_path, map_location=device)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            loaded_epoch = checkpoint['epoch']
            start_epoch = loaded_epoch + 1
            print(f"Resuming training from epoch {start_epoch}")
        else:
            print(f"No checkpoint found for fold {fold_number}, starting training from scratch.")

        # Training loop with progress bar over epochs
        for epoch in tqdm(range(start_epoch, num_epochs + 1),
                          desc=f"Training Fold {fold_number}", unit="epoch"):
            model.train()
            running_loss = 0.0

            # Prepare batch loader without progress bar for batches
            batch_size = 256 # Adjust batch size as needed
            batch_loader_iter = batch_loader(train_files, sample_dir, batch_size=batch_size)

            for batch_samples in batch_loader_iter:
                mol_data_list = []
                pro_data_list = []
                target_list = []

                for sample in batch_samples:
                    mol_data = sample[0]
                    pro_data = sample[1]
                    target = sample[2]

                    mol_data_list.append(mol_data)
                    pro_data_list.append(pro_data)
                    target_list.append(target)

                mol_batch = Batch.from_data_list(mol_data_list).to(device)
                pro_batch = Batch.from_data_list(pro_data_list).to(device)
                target = torch.tensor(target_list, dtype=torch.float32).view(-1).to(device)

                optimizer.zero_grad()
                output = model(mol_batch, pro_batch)
                loss = loss_fn(output.view(-1), target)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            avg_loss = running_loss / len(batch_samples)
            # Use tqdm.write() to print without interfering with the progress bar
            tqdm.write(f"Fold {fold_number}, Epoch {epoch}/{num_epochs}, Loss: {avg_loss:.4f}")

            # Save the model and optimizer states after each epoch
            checkpoint_filename = f"model_fold{fold_number}_epoch{epoch}.pt"
            checkpoint_path = os.path.join(training_model_dir, checkpoint_filename)
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, checkpoint_path)
            tqdm.write(f"Checkpoint saved for fold {fold_number} at epoch {epoch}")

            # Evaluation on the test set after each epoch
            model.eval()
            total_preds, total_labels = [], []
            with torch.no_grad():
                batch_size = 256  # Adjust batch size as needed
                batch_loader_iter = batch_loader(test_files, sample_dir, batch_size=batch_size)

                for batch_samples in batch_loader_iter:
                    mol_data_list = []
                    pro_data_list = []
                    target_list = []

                    for sample in batch_samples:
                        mol_data = sample[0]
                        pro_data = sample[1]
                        target = sample[2]

                        mol_data_list.append(mol_data)
                        pro_data_list.append(pro_data)
                        target_list.append(target)

                    mol_batch = Batch.from_data_list(mol_data_list).to(device)
                    pro_batch = Batch.from_data_list(pro_data_list).to(device)
                    target = torch.tensor(target_list, dtype=torch.float32).view(-1).to(device)

                    output = model(mol_batch, pro_batch)
                    total_preds.append(output.cpu().numpy())
                    total_labels.append(target.cpu().numpy())

            # Convert lists to numpy arrays for evaluation
            total_preds = np.concatenate(total_preds)
            total_labels = np.concatenate(total_labels)

            # Calculate metrics
            mse = get_mse(total_labels, total_preds)
            ci = get_ci(total_labels, total_preds)
            pearson = get_pearson(total_labels, total_preds)

            # Print metrics
            tqdm.write(f"Fold {fold_number}, Epoch {epoch}/{num_epochs} - MSE: {mse:.4f}, CI: {ci:.4f}, Pearson: {pearson:.4f}")

        # Evaluation at the end of training for this fold
        print(f"Final evaluation for Fold {fold_number}: MSE: {mse:.4f}, CI: {ci:.4f}, Pearson: {pearson:.4f}")
        # Store results for this fold
        results.append((mse, ci, pearson))

    return results


if __name__ == "__main__":
    sample_dir = 'prepared_samples'  # Adjust the path to your samples directory
    num_epochs = 250  # Adjust the number of epochs as needed
    n_splits = 5  # Number of folds for cross-validation
    learning_rate = 0.001  # Learning rate

    # Run the training function
    results = train_5fold_cross_validation(sample_dir, num_epochs=num_epochs, n_splits=n_splits, lr=learning_rate)

    # Print overall results
    print("\nCross-validation Results:")
    for fold_idx, (mse, ci, pearson) in enumerate(results):
        print(f"Fold {fold_idx + 1}: MSE={mse:.4f}, CI={ci:.4f}, Pearson={pearson:.4f}")

    # Optionally, compute and print average metrics across folds
    mse_values, ci_values, pearson_values = zip(*results)
    print(f"\nAverage Results:")
    print(f"MSE: {np.mean(mse_values):.4f}")
    print(f"CI: {np.mean(ci_values):.4f}")
    print(f"Pearson Correlation: {np.mean(pearson_values):.4f}")


Running on cpu.
Using existing TrainingModel directory at prepared_samples/TrainingModel

Fold 1/5
GNNNet Loaded
Model is on device: cpu
Loading checkpoint for fold 1 from prepared_samples/TrainingModel/model_fold1_epoch249.pt


KeyError: 'model_state_dict'